# 06 — Findings & Recommendations

**Objective:** Synthesise all analyses into actionable recommendations.

## Structure
1. Executive summary (key numbers)
2. Top 10 opportunity categories with evidence
3. Per-category deep dives (top 3)
4. Cross-country comparison
5. Predictive model: what makes a PL category leader?
6. Limitations and caveats
7. What we'd do with retailer sales data

In [ ]:
from src.models.success_predictor import build_success_predictor

In [ ]:
# TODO: Build final findings narrative